# 9. Exporting Data Example

This notebook shows different ways to export data from memory. This first example exports all conversations from local SQLite memory with their respective score values in a JSON format. The data can currently be exported both as JSON file or a CSV file that will be saved in your results folder within PyRIT. The CSV export is commented out below. In this example, all conversations are exported, but by using other export functions from `memory_interface`, we can export by specific labels and other methods.

In [ ]:
from uuid import uuid4

from pyrit.common.path import DB_DATA_PATH
from pyrit.memory import CentralMemory
from pyrit.models import Message, MessagePiece
from pyrit.setup import IN_MEMORY, initialize_pyrit_async

await initialize_pyrit_async(memory_db_type=IN_MEMORY)  # type: ignore

conversation_id = str(uuid4())

print(conversation_id)

message_list = [
    MessagePiece(
        role="user", original_value="Hi, chat bot! This is my initial prompt.", conversation_id=conversation_id
    ),
    MessagePiece(
        role="assistant", original_value="Nice to meet you! This is my response.", conversation_id=conversation_id
    ),
    MessagePiece(
        role="user",
        original_value="Wonderful! This is my second prompt to the chat bot!",
        conversation_id=conversation_id,
    ),
]

sqlite_memory = CentralMemory.get_memory_instance()
sqlite_memory.add_message_to_memory(request=Message([message_list[0]]))
sqlite_memory.add_message_to_memory(request=Message([message_list[1]]))
sqlite_memory.add_message_to_memory(request=Message([message_list[2]]))

entries = sqlite_memory.get_conversation(conversation_id=conversation_id)

for entry in entries:
    print(entry)

# Define file path for export
json_file_path = DB_DATA_PATH / "conversation_and_scores_json_example.json"
# csv_file_path = DB_DATA_PATH / "conversation_and_scores_csv_example.csv"

# Export the data to a JSON file
conversation_with_scores = sqlite_memory.export_conversations(file_path=json_file_path, export_type="json")
print(f"Exported conversation with scores to JSON: {json_file_path}")

# Export the data to a CSV file
# conversation_with_scores = sqlite_memory.export_conversations(file_path=csv_file_path, export_type="csv")
# print(f"Exported conversation with scores to CSV: {csv_file_path}")

e08d01d5-4447-41fe-a060-c31ecb923381
{}: user: Hi, chat bot! This is my initial prompt.
{}: assistant: Nice to meet you! This is my response.
{}: user: Wonderful! This is my second prompt to the chat bot!
Exported conversation with scores to JSON: C:\git\PyRIT\dbdata\conversation_and_scores_json_example.json


You can also use the exported JSON or CSV files to import the data as a NumPy DataFrame. This can be useful for various data manipulation and analysis tasks.

In [ ]:
import pandas as pd  # type: ignore

df = pd.read_json(json_file_path)
df.head(1)

,id,role,conversation_id,sequence,timestamp,labels,targeted_harm_categories,prompt_metadata,converter_identifiers,prompt_target_identifier,...,original_value_data_type,original_value,original_value_sha256,converted_value_data_type,converted_value,converted_value_sha256,response_error,originator,original_prompt_id,scores
0,90ba18b8-07f7-4092-8d54-55fe0ebb38b4,user,e08d01d5-4447-41fe-a060-c31ecb923381,0,2025-12-05 19:40:38.356295,{},NaN,{},[],{},...,text,"Hi, chat bot! This is my initial prompt.",NaN,text,"Hi, chat bot! This is my initial prompt.",NaN,none,undefined,90ba18b8-07f7-4092-8d54-55fe0ebb38b4,[]
